In [2]:
from scipy.stats import multivariate_normal as mvn
import numpy as np
from tqdm import tqdm

In [224]:
NN = 100 ** 2
N = 8
y = np.ones(N)
covs = []
for _ in range(NN):
    L = np.random.rand(N, N)
    L = L.T @ L
    L = L + np.eye(N)
    covs.append(L)
covs = np.array(covs)

In [247]:
def mvn_multi_log_unnormalized_pdf(y, mus, covs):
    N = len(y)
    covs_inv = np.linalg.inv(covs)
    ret = np.einsum("i,lij,j->l", (-y - mus), covs_inv, (y- mus))
    ret -= np.linalg.slogdet(covs).logabsdet
    # ret -= N * np.log(2 * np.pi)
    # return 0.5 * ret
    return ret

In [248]:
%%timeit
(mvn_multi_log_pdf(y, 0, covs));

29.6 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [207]:
%%time
for i in range(NN):
    (mvn.logpdf(y, cov=covs[i]))

CPU times: user 4min 18s, sys: 28.5 s, total: 4min 47s
Wall time: 2min 24s


In [191]:
cov[0]

array([[1, 1],
       [1, 2]])

In [1]:
# ret = -0.5 * np.einsum("i,ij,j->", (y - 0), np.linalg.inv(cov[i]), (y- 0))
ret = -0.5 * y.T @ np.linalg.inv(cov[i]) @ y
ret -= 0.5 * np.log(np.linalg.det(cov[i]))
ret -= 0.5 * len(y) * np.log(2 * np.pi)
ret

NameError: name 'y' is not defined

In [177]:
np.log(np.e)

1.0

In [4]:
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)

In [5]:
XX, YY = np.meshgrid(x, y)

In [8]:
XX = XX.reshape(-1)
YY = YY.reshape(-1)

In [9]:
XX

array([0.        , 0.01010101, 0.02020202, ..., 0.97979798, 0.98989899,
       1.        ])

In [12]:
YY[None]

array([[0., 0., 0., ..., 1., 1., 1.]])

In [27]:
K = np.array([
    [[1, 3], [3, 4]],
    [[4, 1], [1, 2]],
    [[5, 3], [3, 2]],
])

In [28]:
f = np.array([0,1,2])

In [46]:
def rowpeat(A, M):
    """
    Repeat a rows of A a total of M times.
    """
    ret = np.empty((A.shape[0] * M, *A.shape[1:]), dtype=A.dtype)
    for i in range(A.shape[0]):
        ret[M * i : M * (i + 1)] = A[i]
    return ret


def full_cov(K, J, var_e, var_b):
    """
    Get the covariance matrix of the biases and observations
    """
    N = K.shape[-1]
    N_K = K.shape[0]

    Sigma = np.zeros((N * J + J + N, N * J + J + N))
    Sigma[0 : J * N, 0 : J * N] = np.eye(J * N) * var_e
    Sigma[J * N : J * N + J, J * N : J * N + J] = np.eye(J) * var_b
    Sigma[-N:, -N:] = K
    A = np.zeros((N * J + J, N * J + J + N))
    A[: N * J + J, : N * J + J] = np.eye(N * J + J)

    mix = np.zeros((N * J, J + N))
    mix[:, :J] = np.vstack([np.eye(J) for _ in range(N)])
    mix[:, J:] = rowpeat(np.eye(N), J)
    A[: N * J, N * J :] = mix

    return A @ Sigma @ A.T

In [41]:
J = 3
var_e = 2
var_b = 3
N = K.shape[-1]
N_K = K.shape[0]

In [48]:
Sigma = np.zeros((N_K, N * J + J + N, N * J + J + N))
Sigma[:, 0 : J * N, 0 : J * N] = np.eye(J * N) * var_e
Sigma[:, J * N : J * N + J, J * N : J * N + J] = np.eye(J) * var_b
Sigma[:, -N:, -N:] = K
A = np.zeros((N * J + J, N * J + J + N))
A[: N * J + J, : N * J + J] = np.eye(N * J + J)
mix = np.zeros((N * J, J + N))
mix[:, :J] = np.vstack([np.eye(J) for _ in range(N)])
mix[:, J:] = rowpeat(np.eye(N), J)
A[: N * J, N * J :] = mix

A @ Sigma @ A.T

array([[[ 6.,  1.,  1.,  6.,  3.,  3.,  3.,  0.,  0.],
        [ 1.,  6.,  1.,  3.,  6.,  3.,  0.,  3.,  0.],
        [ 1.,  1.,  6.,  3.,  3.,  6.,  0.,  0.,  3.],
        [ 6.,  3.,  3.,  9.,  4.,  4.,  3.,  0.,  0.],
        [ 3.,  6.,  3.,  4.,  9.,  4.,  0.,  3.,  0.],
        [ 3.,  3.,  6.,  4.,  4.,  9.,  0.,  0.,  3.],
        [ 3.,  0.,  0.,  3.,  0.,  0.,  3.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  3.,  0.,  0.,  3.,  0.],
        [ 0.,  0.,  3.,  0.,  0.,  3.,  0.,  0.,  3.]],

       [[ 9.,  4.,  4.,  4.,  1.,  1.,  3.,  0.,  0.],
        [ 4.,  9.,  4.,  1.,  4.,  1.,  0.,  3.,  0.],
        [ 4.,  4.,  9.,  1.,  1.,  4.,  0.,  0.,  3.],
        [ 4.,  1.,  1.,  7.,  2.,  2.,  3.,  0.,  0.],
        [ 1.,  4.,  1.,  2.,  7.,  2.,  0.,  3.,  0.],
        [ 1.,  1.,  4.,  2.,  2.,  7.,  0.,  0.,  3.],
        [ 3.,  0.,  0.,  3.,  0.,  0.,  3.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  3.,  0.,  0.,  3.,  0.],
        [ 0.,  0.,  3.,  0.,  0.,  3.,  0.,  0.,  3.]],

      

In [49]:
A @ Sigma[0] @ A.T

array([[6., 1., 1., 6., 3., 3., 3., 0., 0.],
       [1., 6., 1., 3., 6., 3., 0., 3., 0.],
       [1., 1., 6., 3., 3., 6., 0., 0., 3.],
       [6., 3., 3., 9., 4., 4., 3., 0., 0.],
       [3., 6., 3., 4., 9., 4., 0., 3., 0.],
       [3., 3., 6., 4., 4., 9., 0., 0., 3.],
       [3., 0., 0., 3., 0., 0., 3., 0., 0.],
       [0., 3., 0., 0., 3., 0., 0., 3., 0.],
       [0., 0., 3., 0., 0., 3., 0., 0., 3.]])

In [17]:
c = np.array([1,2,3])

In [19]:
np.multiply.outer(np.eye(3), c)

array([[1., 2., 3.],
       [0., 0., 0.],
       [0., 0., 0.]])